In [77]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [78]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier

In [79]:
train_df = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")

In [80]:
test_df = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

In [81]:
train_df

In [82]:
test_df

In [83]:
train_df.isnull().sum()

In [84]:
test_df.isnull().sum()

In [85]:
train_df.drop(columns=['location'],inplace=True)
test_df.drop(columns=['location'],inplace=True)

In [86]:
train_df.drop(columns=['id'],inplace=True)
test_df.drop(columns=['id'],inplace=True)

In [87]:
train_df.drop(columns=['keyword'],inplace=True)
test_df.drop(columns=['keyword'],inplace=True)

In [88]:
train_df[train_df['target']==1]

In [89]:
train_df[train_df['target']==0]

In [90]:
train_df['source']="train"
test_df['source']='test'

In [91]:
df = pd.concat([train_df,test_df])

In [92]:
df

In [93]:
lemmatizer = WordNetLemmatizer()
sw = stopwords.words('english')
pstemmer = PorterStemmer()
tokenize = RegexpTokenizer(r'\w+')

In [94]:
sw.extend(['I','u',"the'",'and','It','My','amp'])

In [95]:
def cleantxt(txt):
    tokens = nltk.word_tokenize(txt)
    tokens = tokenize.tokenize(txt)
    clean_tokens = [token for token in tokens if token not in sw]
    lemmatizer_tokens = [lemmatizer.lemmatize(token) for token in clean_tokens]
    stemmed_tokens = [pstemmer.stem(token) for token in lemmatizer_tokens]
    return " ".join(stemmed_tokens)
    
    

In [96]:
df['text'] = df['text'].apply(cleantxt)

In [97]:
df

In [98]:
disaster_text = ' , '.join(str(v) for v in df['text'])
disaster_text = re.sub(r'http','',disaster_text)
no_special_characters = re.sub(r'[^A-Za-z0-9]+',' ',disaster_text)

In [99]:
wordcloud_true = WordCloud().generate(no_special_characters)
plt.figure(figsize=(10,10))
plt.imshow(wordcloud_true)
plt.axis('off')
plt.title("Word Cloud of Real Disaster news")
plt.tight_layout(pad=0)
plt.show()

In [100]:
x = df[df['source']=='train']

In [101]:
x1 = x['text']
y1 = x['target']

In [102]:
from sklearn.model_selection import train_test_split

In [103]:
x_train , x_test , y_train , y_test = train_test_split(x1,y1,test_size=0.2,random_state=42)

In [104]:
from sklearn.pipeline import Pipeline

In [105]:
nb_classifier = Pipeline([('vect',CountVectorizer()),('tfidf',TfidfTransformer()),('clf',MultinomialNB()),])

In [106]:
nb_classifier.fit(x_train,y_train)

In [107]:
ypred = nb_classifier.predict(x_test)

In [108]:
from sklearn.metrics import classification_report

In [110]:
print(classification_report(ypred,y_test))

In [111]:
y = df[df['source']=='test']

In [112]:
dependent = y['target']
independent = y['text']

In [113]:
y_pred = nb_classifier.predict(independent)

In [114]:
y_pred

In [116]:
sample_sub = pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')

In [131]:
sample_sub['target'] = y_pred

In [132]:
sample_sub.head()

In [134]:
sample_sub.to_csv('submission.csv',index=False)